# Directory settings
funnel-transformer-largeを追加

https://www.kaggle.com/competitions/us-patent-phrase-to-phrase-matching/discussion/327288

In [1]:
# ====================================================
# Directory settings
# ====================================================
import os

INPUT_DIR = '../input/us-patent-phrase-to-phrase-matching/'
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# CFG

In [2]:
# ====================================================
# CFG
# ====================================================
class CFG_exp016:
    num_workers=4
    path="../input/pppm-baseline-016/"
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-large"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=133
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]

class CFG_exp026:
    num_workers=4
    path="../input/pppm-baseline-026/"
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-large"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=134
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    
class CFG_exp020:
    num_workers=4
    path="../input/pppm-baseline-020/"
    config_path=path+'config.pth'
    model="roberta-large"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=175
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]

    
class CFG_exp011:
    num_workers=4
    path="../input/pppm-baseline-011/"
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-base"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=133
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]

class CFG_exp018:
    num_workers=4
    path="../input/pppm-baseline-018/"
    config_path=path+'config.pth'
    model="anferico/bert-for-patents"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=117
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    

class CFG_exp032:
    num_workers=4
    path="../input/pppm-baseline-032/"
    config_path=path+'config.pth'
    model="funnel-transformer/large"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=125
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]

# Library

In [3]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import shutil
import string
import pickle
import random
import joblib
import itertools
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
print(f"torch.__version__: {torch.__version__}")
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('pip uninstall -y pytorch-tabnet')

os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels-dataset transformers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels-dataset tokenizers')
os.system('python -m pip install --no-index --find-links=../input/pytorchtabnet/pytorch-tabnet  pytorch-tabnet')

import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Utils

In [4]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score


def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

# OOF

In [5]:
## 別ver
# https://www.kaggle.com/code/jellyz9/tips-for-ensambling
from sklearn.preprocessing import MinMaxScaler #Scaler
from sklearn.linear_model import LinearRegression #重回帰分析
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

use_models =[
    "../input/pppm-baseline-026",
    "../input/pppm-baseline-020",
    "../input/pppm-baseline-018",
    "../input/pppm-baseline-011",
    "../input/pppm-baseline-032"
]

train_df = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/train.csv")
ens_df = train_df[["id", "score"]]

for model in use_models:
  _df = pd.read_pickle(os.path.join(model, "preds", "oof_df.pkl"))
  if len(_df) > len(ens_df):
    print(model)
    _df = _df[_df["flag"]==1]
    df = _df[["id", "pred"]]
    
    MMscaler = MinMaxScaler()
    df["pred"] = MMscaler.fit_transform(df["pred"].values.reshape(-1,1)).reshape(-1)

    df.rename(columns={"pred":model.split("/")[-1]},inplace=True)
    ens_df = pd.merge(ens_df, df, on="id", how="left")

  else:
    df = _df[["id", "pred"]]

    MMscaler = MinMaxScaler()
    df["pred"] = MMscaler.fit_transform(df["pred"].values.reshape(-1,1)).reshape(-1)

    df.rename(columns={"pred":model.split("/")[-1]},inplace=True)
    ens_df = pd.merge(ens_df, df, on="id", how="left")

ens_df


In [6]:
# アンサンブル

Y = ens_df["score"]
X = ens_df.drop(["score","id"], axis=1)

plt.figure(figsize=(8,8))
colormap = plt.cm.RdBu
sns.heatmap(ens_df.drop("id",axis=1).astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)

# mdoel
models = []
ens_scores = []
va_idxes = []
preds = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)
for i, (tr_idx, val_idx) in enumerate(list(kf.split(X, Y))):
    tr_x, tr_y = X.values[tr_idx], Y.values[tr_idx]
    val_x, val_y = X.values[val_idx], Y.values[val_idx]
    
    model = LinearRegression()
    model.fit(tr_x, tr_y)
    pred = model.predict(val_x)
    score = get_score(pred, val_y)
    
    
    models.append(model)
    ens_scores.append(score)
    va_idxes.append(val_idx)
    preds.append(pred)

va_idxes = np.concatenate(va_idxes)
order = np.argsort(va_idxes)
preds = np.concatenate(preds, axis=0)
preds = preds[order]

final_score = get_score(Y, preds)
print(final_score)
LOGGER.info(f'CV Score: {final_score:<.4f}')

# stacking oof

In [7]:
stacking_dir = "../input/5model-linear-stacking/5model_kfsky_linear"
dirs = os.listdir(stacking_dir)
ens_df = []

for dir in dirs:
    path = os.path.join(stacking_dir, dir, "oof_df.csv")
    print(path)
    df = pd.read_csv(path)
    ens_df.append(df["score"].values)
    
ens_df = pd.DataFrame(ens_df).T
ens_df[["id", "score"]] =  train_df[["id", "score"]]
print(ens_df.shape)
ens_df.head()

In [8]:
# アンサンブル

# Y = ens_df["score"]
X = ens_df.drop(["score","id"], axis=1)

# plt.figure(figsize=(8,8))
# colormap = plt.cm.RdBu
# sns.heatmap(ens_df.drop("id",axis=1).astype(float).corr(),linewidths=0.1,vmax=1.0, 
#             square=True, cmap=colormap, linecolor='white', annot=True)

# mdoel
models = []
ens_scores = []
va_idxes = []
preds = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)
for i, (tr_idx, val_idx) in enumerate(list(kf.split(X, Y))):
    tr_x, tr_y = X.values[tr_idx], Y.values[tr_idx]
    val_x, val_y = X.values[val_idx], Y.values[val_idx]
    
    model = LinearRegression()
    model.fit(tr_x, tr_y)
    pred = model.predict(val_x)
    score = get_score(pred, val_y)
    
    
    models.append(model)
    ens_scores.append(score)
    va_idxes.append(val_idx)
    preds.append(pred)

va_idxes = np.concatenate(va_idxes)
order = np.argsort(va_idxes)
preds = np.concatenate(preds, axis=0)
preds = preds[order]

final_score = get_score(Y, preds)
print(final_score)
LOGGER.info(f'CV Score: {final_score:<.4f}')

# Data Loading

In [9]:
# ====================================================
# Data Loading
# ====================================================
test = pd.read_csv(INPUT_DIR+'test.csv')
submission = pd.read_csv(INPUT_DIR+'sample_submission.csv')
print(f"test.shape: {test.shape}")
print(f"submission.shape: {submission.shape}")
display(test.head())
display(submission.head())

In [10]:
# ====================================================
# CPC Data
# ====================================================

def omit_char(x):
  x = x.replace(";", "") # ; を削除
  x = x.replace("[", "") # [] を削除
  x = x.replace("]", "") # [] を削除
  x = x.lower() # すべて小文字に変換
  return x

cpc_texts = torch.load(os.path.join(CFG_exp016.path, "cpc_texts.pth"))
test['context_text'] = test['context'].map(cpc_texts)
test['anchor'].replace("dry coating composition1", "dry coating composition", inplace=True)


test['context_text'] = test['context_text'].map(omit_char)
test['context_text'] = test['context_text'].replace("human necessities. griculture forestry animal husbandry hunting trapping fishing", 
                                                      "human necessities. agriculture forestry animal husbandry hunting trapping fishing")
test['context_text'] = test['context_text'].str.replace("hemistry", "chemistry")
display(test.head())

In [11]:
test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
display(test.head())

# exp026

## tokenizer

In [12]:
# ====================================================
# tokenizer
# ====================================================
CFG_exp026.tokenizer = AutoTokenizer.from_pretrained(CFG_exp026.path+'tokenizer/')

## Dataset

In [13]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text):
    inputs = cfg.tokenizer(text,
                           add_special_tokens=True,
                           max_length=cfg.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['text'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        return inputs

## Model

In [14]:
# ====================================================
# Model
# ====================================================
class CustomModel_exp026(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, self.config.hidden_size),
            nn.LayerNorm(self.config.hidden_size),
            nn.GELU(),
            nn.Linear(self.config.hidden_size, 1),
            nn.Softmax(dim=1)
        )
        self.fc_dropout1 = nn.Dropout(0.1)
        self.fc_dropout2 = nn.Dropout(0.2)
        self.fc_dropout3 = nn.Dropout(0.3)
        self.fc_dropout4 = nn.Dropout(0.4)
        self.fc_dropout5 = nn.Dropout(0.5)
        
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)

        feature1 = self.fc_dropout1(feature)
        feature2 = self.fc_dropout2(feature)
        feature3 = self.fc_dropout3(feature)
        feature4 = self.fc_dropout4(feature)
        feature5 = self.fc_dropout5(feature)

        feature_all = (feature1+feature2+feature3+feature4+feature5)/5
        output = self.fc(feature_all)
        
        return output

In [15]:
# ====================================================
# inference
# ====================================================
def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [ ]:
test_dataset = TestDataset(CFG_exp026, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG_exp026.batch_size,
                         shuffle=False,
                         num_workers=CFG_exp026.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in CFG_exp026.trn_fold:
    model = CustomModel_exp026(CFG_exp026, config_path=CFG_exp026.config_path, pretrained=False)
    state = torch.load(os.path.join(CFG_exp026.path,"model" , f"{CFG_exp026.model.replace('/', '-')}_fold{fold}_best.pth"),
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_exp026 = np.mean(predictions, axis=0)

# exp020

In [ ]:
# ====================================================
# tokenizer
# ====================================================
CFG_exp020.tokenizer = AutoTokenizer.from_pretrained(CFG_exp020.path+'tokenizer/')

In [ ]:
# ====================================================
# Model
# ====================================================
class CustomModel_exp020(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, self.config.hidden_size),
            nn.LayerNorm(self.config.hidden_size),
            nn.GELU(),
            nn.Linear(self.config.hidden_size, 1),
            nn.Softmax(dim=1)
        )
        self.fc_dropout1 = nn.Dropout(0.1)
        self.fc_dropout2 = nn.Dropout(0.2)
        self.fc_dropout3 = nn.Dropout(0.3)
        self.fc_dropout4 = nn.Dropout(0.4)
        self.fc_dropout5 = nn.Dropout(0.5)
        
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)

        feature1 = self.fc_dropout1(feature)
        feature2 = self.fc_dropout2(feature)
        feature3 = self.fc_dropout3(feature)
        feature4 = self.fc_dropout4(feature)
        feature5 = self.fc_dropout5(feature)

        feature_all = (feature1+feature2+feature3+feature4+feature5)/5
        output = self.fc(feature_all)
        
        return output

In [ ]:
test_dataset = TestDataset(CFG_exp020, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG_exp020.batch_size,
                         shuffle=False,
                         num_workers=CFG_exp020.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in CFG_exp020.trn_fold:
    model = CustomModel_exp020(CFG_exp020, config_path=CFG_exp020.config_path, pretrained=False)
    state = torch.load(os.path.join(CFG_exp020.path,"model" , f"{CFG_exp020.model.replace('/', '-')}_fold{fold}_best.pth"),
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_exp020 = np.mean(predictions, axis=0)

# exp011

In [ ]:
# ====================================================
# tokenizer
# ====================================================
CFG_exp011.tokenizer = AutoTokenizer.from_pretrained(CFG_exp011.path+'tokenizer/')

In [ ]:
# ====================================================
# Model
# ====================================================
class CustomModel_exp011(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        self.fc_dropout1 = nn.Dropout(0.1)
        self.fc_dropout2 = nn.Dropout(0.2)
        self.fc_dropout3 = nn.Dropout(0.3)
        self.fc_dropout4 = nn.Dropout(0.4)
        self.fc_dropout5 = nn.Dropout(0.5)
        
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)

        feature1 = self.fc_dropout1(feature)
        feature2 = self.fc_dropout2(feature)
        feature3 = self.fc_dropout3(feature)
        feature4 = self.fc_dropout4(feature)
        feature5 = self.fc_dropout5(feature)

        feature_all = (feature1+feature2+feature3+feature4+feature5)/5
        output = self.fc(feature_all)
        
        return output

In [ ]:
test_dataset = TestDataset(CFG_exp011, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG_exp011.batch_size,
                         shuffle=False,
                         num_workers=CFG_exp011.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in CFG_exp011.trn_fold:
    model = CustomModel_exp011(CFG_exp011, config_path=CFG_exp011.config_path, pretrained=False)
    state = torch.load(os.path.join(CFG_exp011.path,"model" , f"{CFG_exp011.model.replace('/', '-')}_fold{fold}_best.pth"),
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_exp011 = np.mean(predictions, axis=0)

# exp018

In [ ]:
# ====================================================
# tokenizer
# ====================================================
CFG_exp018.tokenizer = AutoTokenizer.from_pretrained(CFG_exp018.path+'tokenizer/')

In [ ]:
# ====================================================
# Model
# ====================================================
class CustomModel_exp018(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, self.config.hidden_size),
            nn.LayerNorm(self.config.hidden_size),
            nn.GELU(),
            nn.Linear(self.config.hidden_size, 1),
            nn.Softmax(dim=1)
        )
        self.fc_dropout1 = nn.Dropout(0.1)
        self.fc_dropout2 = nn.Dropout(0.2)
        self.fc_dropout3 = nn.Dropout(0.3)
        self.fc_dropout4 = nn.Dropout(0.4)
        self.fc_dropout5 = nn.Dropout(0.5)
        
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)

        feature1 = self.fc_dropout1(feature)
        feature2 = self.fc_dropout2(feature)
        feature3 = self.fc_dropout3(feature)
        feature4 = self.fc_dropout4(feature)
        feature5 = self.fc_dropout5(feature)

        feature_all = (feature1+feature2+feature3+feature4+feature5)/5
        output = self.fc(feature_all)
        
        return output

In [ ]:
test_dataset = TestDataset(CFG_exp018, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG_exp018.batch_size,
                         shuffle=False,
                         num_workers=CFG_exp018.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in CFG_exp018.trn_fold:
    model = CustomModel_exp018(CFG_exp018, config_path=CFG_exp018.config_path, pretrained=False)
    state = torch.load(os.path.join(CFG_exp018.path,"model" , f"{CFG_exp018.model.replace('/', '-')}_fold{fold}_best.pth"),
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_exp018 = np.mean(predictions, axis=0)

# exp032

In [ ]:
# ====================================================
# tokenizer
# ====================================================
CFG_exp032.tokenizer = AutoTokenizer.from_pretrained(CFG_exp032.path+'tokenizer/')

In [ ]:
# ====================================================
# Model
# ====================================================
class CustomModel_exp032(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, self.config.hidden_size),
            nn.LayerNorm(self.config.hidden_size),
            nn.GELU(),
            nn.Linear(self.config.hidden_size, 1),
            nn.Softmax(dim=1)
        )
        self.fc_dropout1 = nn.Dropout(0.1)
        self.fc_dropout2 = nn.Dropout(0.2)
        self.fc_dropout3 = nn.Dropout(0.3)
        self.fc_dropout4 = nn.Dropout(0.4)
        self.fc_dropout5 = nn.Dropout(0.5)
        
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)

        feature1 = self.fc_dropout1(feature)
        feature2 = self.fc_dropout2(feature)
        feature3 = self.fc_dropout3(feature)
        feature4 = self.fc_dropout4(feature)
        feature5 = self.fc_dropout5(feature)

        feature_all = (feature1+feature2+feature3+feature4+feature5)/5
        output = self.fc(feature_all)
        
        return output

In [ ]:
test_dataset = TestDataset(CFG_exp032, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG_exp032.batch_size,
                         shuffle=False,
                         num_workers=CFG_exp032.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in CFG_exp032.trn_fold:
    model = CustomModel_exp032(CFG_exp032, config_path=CFG_exp032.config_path, pretrained=False)
    state = torch.load(os.path.join(CFG_exp032.path,"model" , f"{CFG_exp032.model.replace('/', '-')}_fold{fold}_best.pth"),
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_exp032 = np.mean(predictions, axis=0)

# Submission

In [ ]:
# ensamble

MMscaler = MinMaxScaler()

predictions_exp026 = MMscaler.fit_transform(predictions_exp026.reshape(-1,1)).reshape(-1)
predictions_exp020 = MMscaler.fit_transform(predictions_exp020.reshape(-1,1)).reshape(-1)
predictions_exp011 = MMscaler.fit_transform(predictions_exp011.reshape(-1,1)).reshape(-1)
predictions_exp018 = MMscaler.fit_transform(predictions_exp018.reshape(-1,1)).reshape(-1)
predictions_exp032 = MMscaler.fit_transform(predictions_exp032.reshape(-1,1)).reshape(-1)

use_models =[
    "../input/pppm-baseline-026",
    "../input/pppm-baseline-020",
    "../input/pppm-baseline-018",
    "../input/pppm-baseline-011",
    "../input/pppm-baseline-032"
]

prediction_lists = [
    predictions_exp026,
    predictions_exp020,
    predictions_exp018,
    predictions_exp011,
    predictions_exp032
]

data_dir = {}
for model_name, p in zip(use_models, prediction_lists):
    data_dir[model_name.split("/")[-1]] = p
    

prediction_df = pd.DataFrame(data_dir)
# final_predictions = []
# # 回帰モデル
# for model in models:
#     prediction = model.predict(prediction_df)
#     final_predictions.append(prediction)
# final_prediction = np.mean(final_predictions,axis=0)
# final_prediction
# #final_predictions =  predictions_deberta * round(deberta_weight, 2) + predictions_bert_patents * round(bert_patents_weight, 2)

# stacking

In [ ]:
display(prediction_df)
a

In [ ]:
# test_data = pd.DataFrame(final_predictions).T
test_data = pd.DataFrame(final_predictions).T

test_data.columns = [f"pred{i}" for i in range(len(test_data.columns))]
features = test_data.columns
test_data.head()

In [ ]:
class stacking_CFG:
    max_grad_norm=1000
    gradient_accumulation_steps=1
    hidden_size=256
    dropout=0.3
    lr=1e-4
    batch_size=128
    epochs=50
    weight_decay=1e-5
    n_fold = 5
    features = test.columns
    model_dir = "../input/5model-linear-stacking/5model_kfsky_linear/"
    model_path = None

In [ ]:
class StackingDataset(Dataset):
    def __init__(self, df, features, labels=None, model_name=None):
        self.df = df[features].values
        if model_name == "2dcnn":
            # [N, Models, Labels, Channel] -> [N, Channel, Models, Labels]
            self.df = self.df.reshape(-1, len(features), 1, 1)
            self.df = self.df.transpose(0,3,1,2)
        if not labels is None:
            self.labels = labels.values
        else:
            self.labels = labels
        
    def __len__(self, ):
        return len(self.df)
    
    
    def __getitem__(self, item):
        inputs =  torch.FloatTensor(self.df[item]).float()
        
        if self.labels is None:
            return inputs
        labels = torch.tensor(self.labels[item]).float()
        return inputs, labels

## 1dcnn

In [ ]:
class oneDCNN(nn.Module):

        def __init__(self, num_features, num_targets, hidden_size, cfg=None):
            self. model_name="1dcnn"
            super(oneDCNN, self).__init__()
            hidden_size = 1024
            cha_1 = 256
            cha_2 = 512
            cha_3 = 512

            cha_1_reshape = int(hidden_size/cha_1)
            cha_po_1 = int(hidden_size/cha_1/2)
            cha_po_2 = int(hidden_size/cha_1/2/2) * cha_3

            self.cha_1 = cha_1
            self.cha_2 = cha_2
            self.cha_3 = cha_3
            self.cha_1_reshape = cha_1_reshape
            self.cha_po_1 = cha_po_1
            self.cha_po_2 = cha_po_2

            self.batch_norm1 = nn.BatchNorm1d(num_features)
            self.dropout1 = nn.Dropout(0.1)
            self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))

            self.batch_norm_c1 = nn.BatchNorm1d(cha_1)
            self.dropout_c1 = nn.Dropout(0.1)
            self.conv1 = nn.utils.weight_norm(nn.Conv1d(cha_1,cha_2, kernel_size = 5, stride = 1, padding=2,  bias=False),dim=None)

            self.ave_po_c1 = nn.AdaptiveAvgPool1d(output_size = cha_po_1)

            self.batch_norm_c2 = nn.BatchNorm1d(cha_2)
            self.dropout_c2 = nn.Dropout(0.1)
            self.conv2 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)

            self.batch_norm_c2_1 = nn.BatchNorm1d(cha_2)
            self.dropout_c2_1 = nn.Dropout(0.3)
            self.conv2_1 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)

            self.batch_norm_c2_2 = nn.BatchNorm1d(cha_2)
            self.dropout_c2_2 = nn.Dropout(0.2)
            self.conv2_2 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_3, kernel_size = 5, stride = 1, padding=2, bias=True),dim=None)

            self.max_po_c2 = nn.MaxPool1d(kernel_size=4, stride=2, padding=1)

            self.flt = nn.Flatten()

            self.batch_norm3 = nn.BatchNorm1d(cha_po_2)
            self.dropout3 = nn.Dropout(0.2)
            self.dense3 = nn.utils.weight_norm(nn.Linear(cha_po_2, num_targets))

        def forward(self, x):

            x = self.batch_norm1(x)
            x = self.dropout1(x)
            x = F.celu(self.dense1(x), alpha=0.06)

            x = x.reshape(x.shape[0],self.cha_1,
                          self.cha_1_reshape)

            x = self.batch_norm_c1(x)
            x = self.dropout_c1(x)
            x = F.relu(self.conv1(x))

            x = self.ave_po_c1(x)

            x = self.batch_norm_c2(x)
            x = self.dropout_c2(x)
            x = F.relu(self.conv2(x))
            x_s = x

            x = self.batch_norm_c2_1(x)
            x = self.dropout_c2_1(x)
            x = F.relu(self.conv2_1(x))

            x = self.batch_norm_c2_2(x)
            x = self.dropout_c2_2(x)
            x = F.relu(self.conv2_2(x))
            x =  x * x_s

            x = self.max_po_c2(x)

            x = self.flt(x)

            x = self.batch_norm3(x)
            x = self.dropout3(x)
            x = self.dense3(x)

            return x

In [ ]:
def inference_fn(test_loader, model, device):

    model.eval()
    preds = []

    for step, (x) in enumerate(test_loader):

        x = x.to(device, non_blocking=True)

        with torch.no_grad():
            pred = model(x)

        preds.append(pred.sigmoid().detach().cpu().numpy())
#         preds.append(pred.tanh().detach().cpu().numpy())

    preds = np.concatenate(preds)

    return preds

In [ ]:
from tqdm.notebook import tqdm
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
import io
import zipfile
def run_single_nn(cfg, test, features, device, fold_num, seed, MODEL,save_dir,like_sklearn=False):
    seed_everything(seed=seed)
    
    if like_sklearn:
        model = MODEL()
        model_path = io.BytesIO()
        with zipfile.ZipFile(model_path, "w") as z:
            z.write(f"{save_dir}tabnet_fold{fold_num}_seed{seed}/model_params.json", arcname="model_params.json")
            z.write(f"{save_dir}tabnet_fold{fold_num}_seed{seed}/network.pt", arcname="network.pt")
        model.load_model(model_path)

        predictions = model.predict(test[features].values)
    else:
        model = MODEL(
                num_features=len(features), num_targets=1, hidden_size=cfg.hidden_size, cfg=cfg
        )

        model.load_state_dict(torch.load(f"{save_dir}{model.model_name}_fold{fold_num}_seed{seed}.pth",map_location="cpu"), )
        model.to(device, non_blocking=True)

        test_dataset = StackingDataset(test, features, None, model.model_name)

        test_loader = DataLoader(test_dataset, batch_size=cfg.batch_size, shuffle=False, 
                                  num_workers=4, pin_memory=False, drop_last=False)


        # inference
        predictions = inference_fn(test_loader, model, device)     
    return predictions
                          

def run_kfold_nn(cfg, test, features, device, n_fold, seed,MODEL, save_dir, like_sklearn=False):
    oof = np.zeros((len(test), 1))
    for fold_num in range(n_fold):
        LOGGER.info(f"fold {fold_num}")
        _oof = run_single_nn(cfg, test, features, device, fold_num, seed, MODEL,save_dir,like_sklearn)
        oof += _oof
    return oof/n_fold

In [ ]:
stacking_CFG.model_path = stacking_CFG.model_dir+"1dcnn/"
predictions = np.zeros((len(test), 1))
SEED = [42, 1999, 2022]
for i, seed in enumerate(SEED):
    print(seed)
    _pred = run_kfold_nn(stacking_CFG, test_data, features, device,
                        n_fold=5, seed=seed, MODEL=oneDCNN, save_dir=stacking_CFG.model_path,
                         )
    predictions += _pred/len(SEED)
    
predictions_1dcnn = predictions

# 2dcnn

In [ ]:
class twoDCNN(nn.Module):#input5
    def __init__(self, num_features, num_targets, hidden_size, cfg):
        self.model_name = "2dcnn"
        self.num_features = num_features
        self.num_targets = num_targets
        super(twoDCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3, 1), bias=False)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(2, 1), bias=False)
        self.relu2 = nn.ReLU()
        
        # self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(2, 1), bias=False)
        # self.relu3 = nn.ReLU()
        # self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 1), bias=False)
        # self.relu4 = nn.ReLU()
        
#         self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(2, 1), bias=False)
#         self.relu5 = nn.ReLU()
#         self.conv6 = nn.Conv2d(in_channels=128, out_channels=1024, kernel_size=(1, 1), bias=False)
#         self.relu6 = nn.ReLU()
            
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_features=32, out_features=512)
        # self.fc1 = nn.Linear(in_features=80, out_features=512)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(in_features=512, out_features=num_targets)
        

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        
#         x = self.conv3(x)
#         x = self.relu3(x)
#         x = self.conv4(x)
#         x = self.relu4(x)
        
#         x = self.conv5(x)
#         x = self.relu5(x)
#         x = self.conv6(x)
#         x = self.relu6(x)

        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x

In [ ]:
stacking_CFG.model_path = stacking_CFG.model_dir+"2dcnn/"
predictions = np.zeros((len(test), 1))
SEED = [42, 1999, 2022]
for i, seed in enumerate(SEED):
    print(seed)
    _pred = run_kfold_nn(stacking_CFG, test_data, features, device,
                        n_fold=5, seed=seed, MODEL=twoDCNN, save_dir=stacking_CFG.model_path,
                         )
    predictions += _pred/len(SEED)
    
predictions_2dcnn = predictions

# MLP

In [ ]:
class StackingMLP(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size, cfg):
        super().__init__()
        self.model_name = "mlp"
        cfg.num_features = num_features
        cfg.hidden_size = hidden_size
    
        self.mlp = nn.Sequential(
            nn.BatchNorm1d((num_features)),
            nn.Dropout(cfg.dropout),
            nn.utils.weight_norm(nn.Linear((cfg.num_features), cfg.hidden_size)),
            nn.PReLU(),

            
            nn.BatchNorm1d(cfg.hidden_size),
            nn.Dropout(cfg.dropout),
            nn.utils.weight_norm(nn.Linear(cfg.hidden_size, cfg.hidden_size)),
            nn.PReLU(),

                        
            nn.BatchNorm1d(cfg.hidden_size),
            nn.Dropout(cfg.dropout),
            nn.utils.weight_norm(nn.Linear(cfg.hidden_size, num_targets)),

        )
        
        self.shallow_mlp = nn.Sequential(
        
                          nn.Linear((cfg.num_features), cfg.hidden_size),
                          nn.BatchNorm1d(cfg.hidden_size),
                          nn.Dropout(cfg.dropout),
                          nn.PReLU(),
                          nn.Linear(cfg.hidden_size, cfg.hidden_size),
                          nn.BatchNorm1d(cfg.hidden_size),
                          nn.Dropout(cfg.dropout),
                          nn.PReLU(),
                          nn.Linear(cfg.hidden_size, num_targets),
                          )


    def forward(self, x):
        # if "shallow" in  stacking_exp_name:
        x = self.shallow_mlp(x)
        # else:
        #     x = self.mlp(x)
        return x

In [ ]:
stacking_CFG.model_path = stacking_CFG.model_dir+"mlp/"
predictions = np.zeros((len(test), 1))
stacking_exp_name="shallow"
SEED = [42, 1999, 2022]
for i, seed in enumerate(SEED):
    print(seed)
    _pred = run_kfold_nn(stacking_CFG, test_data, features, device,
                        n_fold=5, seed=seed, MODEL=StackingMLP, save_dir=stacking_CFG.model_path,
                         )
    predictions += _pred/len(SEED)
predictions_mlp = predictions

# GCN

In [ ]:
import typing as tp

class MLP(nn.Module):
    """Stacked Dense layers"""
    
    def __init__(
        self, n_features_list: tp.List[int], use_tail_as_out: bool=False,
        drop_rate: float=0.0, use_bn: bool=False, use_wn: bool=False,
        activ:str="relu", block_name: str="LBAD"
    ):
        """"""
        super(MLP, self).__init__()
        n_layers = len(n_features_list) - 1
        block_class = {
            "LBAD": LBAD, "BDLA": BDLA, "LABD": LABD}[block_name]
        layers = []
        for i in range(n_layers):
            in_feats, out_feats = n_features_list[i: i + 2]
            if i == n_layers - 1 and use_tail_as_out:
                if block_name in ["BDLA"]:
                    layer = block_class(in_feats, out_feats, drop_rate, use_bn,  use_wn, "identity")
                else:
                    layer = nn.Linear(in_feats, out_feats)
                    if use_wn:
                        layer = nn.utils.weight_norm(layer)
            else:
                layer = block_class(in_feats, out_feats, drop_rate, use_bn,  use_wn, activ)
            layers.append(layer)
                
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """"""
        return self.layers(x)


class CNNStacking1d(nn.Module):
    """1D-CNN for Stacking."""
    
    def __init__(
        self, n_models: int,
        n_channels_list: tp.List[int], use_bias: bool=False,
        kwargs_head: tp.Dict={},
    ):
        """"""
        super(CNNStacking1d, self).__init__()
        self.n_conv_layers = len(n_channels_list) - 1
        for i in range(self.n_conv_layers):
            in_ch = n_channels_list[i]
            out_ch = n_channels_list[i + 1]
            layer = nn.Sequential(
                nn.Conv1d(
                    in_ch, out_ch, kernel_size=3, stride=1, padding=0, bias=use_bias),
                # nn.BatchNorm1d(out_ch),
                nn.ReLU(inplace=True))
            setattr(self, "conv{}".format(i + 1), layer)
        
        kwargs_head["n_features_list"][0] = (n_models - 2 * self.n_conv_layers) * n_channels_list[-1]
        self.head = MLP(**kwargs_head)
    
    def forward(self, x: torch.FloatTensor) -> torch.Tensor:
        """"""
        bs = x.shape[0]
        h = x  # shape: (bs, n_classes, n_models)
        for i in range(self.n_conv_layers):
            h = getattr(self, "conv{}".format(i + 1))(h)
            
        h = torch.reshape(h, (bs, -1))
        h = self.head(h)
        return h
    
    
    
def get_activation(activ_name: str="relu"):
    """"""
    act_dict = {
        "relu": nn.ReLU(),
        "tanh": nn.Tanh(),
        "sigmoid": nn.Sigmoid(),
        "identity": nn.Identity()}
    if activ_name in act_dict:
        return act_dict[activ_name]
    elif re.match(r"^htanh\_\d{4}$", activ_name):
        bound = int(activ_name[-4:]) / 1000
        return nn.Hardtanh(-bound, bound)
    else:
        raise NotImplementedError

class LBAD(nn.Module):
    """Linear (-> BN) -> Activation (-> Dropout)"""
    
    def __init__(
        self, in_features: int, out_features: int, drop_rate: float=0.0,
        use_bn: bool=False, use_wn: bool=False, activ: str="relu"
    ):
        """"""
        super(LBAD, self).__init__()
        layers = [nn.Linear(in_features, out_features)]
        if use_wn:
            layers[0] = nn.utils.weight_norm(layers[0])
        
        if use_bn:
            layers.append(nn.BatchNorm1d(out_features))
        
        layers.append(get_activation(activ))
        
        if drop_rate > 0:
            layers.append(nn.Dropout(drop_rate))
        
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """"""
        return self.layers(x)
    
    
class BDLA(nn.Module):
    """(BN -> Dropout ->) Linear -> Activation"""
    
    def __init__(
        self, in_features: int, out_features: int, drop_rate: float=0.0,
        use_bn: bool=False, use_wn: bool=False, activ: str="relu"
    ):
        """"""
        super(BDLA, self).__init__()
        layers = []
        if use_bn:
            layers.append(nn.BatchNorm1d(in_features))
            
        if drop_rate > 0:
            layers.append(nn.Dropout(drop_rate))
        
        layers.append(nn.Linear(in_features, out_features))
        if use_wn:
            layers[-1] = nn.utils.weight_norm(layers[-1])
            
        layers.append(get_activation(activ))
        
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """"""
        return self.layers(x)
    

class LABD(nn.Module):
    """Linear -> Activation (-> BN -> Dropout) """
    
    def __init__(
        self, in_features: int, out_features: int, drop_rate: float=0.0,
        use_bn: bool=False, use_wn: bool=False, activ: str="relu"
    ):
        """"""
        super(LABD, self).__init__()
        layers = [nn.Linear(in_features, out_features), get_activation(activ)]
        
        if use_wn:
            layers[0] = nn.utils.weight_norm(layers[0])
        
        if use_bn:
            layers.append(nn.BatchNorm1d(out_features))
        
        if drop_rate > 0:
            layers.append(nn.Dropout(drop_rate))
        
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """"""
        return self.layers(x)
# # for GCNs
def vector_wise_matmul(X: torch.Tensor, W: torch.Tensor) -> torch.Tensor:
    """
    See input matrixes X as bags of vectors, and multiply corresponding weight matrices by vector.
    
    Args:
        X: Input Tensor, shape: (batch_size, **n_vectors**, in_features)
        W: Weight Tensor, shape: (**n_vectors**, out_features, in_features)
    """

    X = torch.transpose(X, 0, 1)  # shape: (n_vectors, batch_size, in_features)
    W = torch.transpose(W, 1, 2)  # shape: (n_vectors, in_features, out_features)
    H = torch.matmul(X, W)        # shape: (n_vectors, batch_size, out_features)
    H = torch.transpose(H, 0, 1)  # shape: (batch_size, n_vectors, out_features)
    
    return H


def vector_wise_shared_matmul(X: torch.Tensor, W: torch.Tensor) -> torch.Tensor:
    """
    See input matrixes X as bags of vectors, and multiply **shared** weight matrices.
    
    Args:
        X: Input Tensor, shape: (batch_size, **n_vectors**, in_features)
        W: Weight Tensor, shape: (out_features, in_features)
    """
    # W = torch.transpose(W, 0, 1)  # shape: (in_features, out_features)
    # H = torch.matmul(X, W)        # shape: (batch_size, n_vectors, out_features)
    
    H = nn.functional.linear(X, W)  # shape: (batch_size, n_vectors, out_features)
    
    return H
def _calculate_fan_in_and_fan_out_for_vwl(tensor) -> tp.Tuple[int]:
    """
    Input tensor: (n_vectors, out_features, in_features) or (out_features, in_features)
    """
    dimensions = tensor.dim()
    if dimensions < 2:
        raise ValueError("Fan in and fan out can not be computed for tensor with fewer than 2 dimensions")

    fan_in = tensor.size(-1)
    fan_out = tensor.size(-2)

    return fan_in, fan_out
    

def _calculate_correct_fan_for_vwl(tensor, mode) -> int:
    """"""
    mode = mode.lower()
    valid_modes = ['fan_in', 'fan_out']
    if mode not in valid_modes:
        raise ValueError("Mode {} not supported, please use one of {}".format(mode, valid_modes))

    fan_in, fan_out = _calculate_fan_in_and_fan_out_for_vwl(tensor)
    return fan_in if mode == 'fan_in' else fan_out


def kaiming_uniform_for_vwl(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    """"""
    fan = _calculate_correct_fan_for_vwl(tensor, mode)
    gain = nn.init.calculate_gain(nonlinearity, a)
    std = gain / np.sqrt(fan)
    bound = np.sqrt(3.0) * std  # Calculate uniform bounds from standard deviation
    with torch.no_grad():
        return tensor.uniform_(-bound, bound)
class VectorWiseLinear(nn.Module):
    """
    For mini batch which have several matrices,
    see as these matrixes as bags of vectors, and multiply weight matrices by vector.
    
    input    X: (batch_size, **n_vectors**, in_features)
    weight W: (**n_vector**, out_features, in_features)
    output  Y: (batch_size, **n_vectors**, out_features)

    **Note**: For simplicity, bias is not described.
    
    X and W are can be seen as below.
    X: [
            [vec_{ 1, 1}, vec_{ 1, 2}, ... vec_{ 1, n_vectors}],
            [vec_{ 2, 1}, vec_{ 2, 2}, ... vec_{ 2, n_vectors}],
                                            .
                                            .
            [vec_{bs, 1}, vec_{bs, 2}, ... vec_{bs, n_vectors}]
        ]
    W: [
            Mat_{1}, Mat_{2}, ... , Mat_{n_vectors}
        ]
    Then Y is calclauted as:
    Y: [
        [ Mat_{1} vec_{ 1, 1}, Mat_{2} vec_{ 1, 2}, ... Mat_{n_vectors} vec_{ 1, n_vectors}],
        [ Mat_{1} vec_{ 2, 1}, Mat_{2} vec_{ 2, 2}, ... Mat_{n_vectors} vec_{ 2, n_vectors}],
        .
        .
        [ Mat_{1} vec_{bs, 1}, Mat_{2} vec_{bs, 2}, ... Mat_{n_vectors} vec_{bs, n_vectors}],
    ]
    """
    
    def __init__(
        self,
        in_features: int, out_features: int, n_vectors: int,
        bias: bool=True, weight_shared: bool=True
    ) -> None:
        """Initialize."""
        super(VectorWiseLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.n_vectors = n_vectors
        self.weight_shared = weight_shared
        
        if self.weight_shared:
            self.weight = nn.Parameter(
                torch.Tensor(self.out_features, self.in_features)).to(device)
            self.matmul_func = vector_wise_shared_matmul
        else:
            self.weight = nn.Parameter(
                torch.Tensor(self.n_vectors, self.out_features, self.in_features))
            self.matmul_func = vector_wise_matmul
            
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
            
        self.reset_parameters()
        
    def reset_parameters(self) -> None:
        """Initialize weight and bias."""
        kaiming_uniform_for_vwl(self.weight, a=np.sqrt(5))
        if self.bias is not None:
            fan_in, _ = _calculate_fan_in_and_fan_out_for_vwl(self.weight)
            bound = 1 / np.sqrt(fan_in)
            nn.init.uniform_(self.bias, -bound, bound)
             
    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """Forward."""
        H = self.matmul_func(X, self.weight)
        if self.bias is not None:
            H = H + self.bias
        
        return H
class GraphConv(nn.Module):
    """Basic Graph Convolution Layer."""
    
    def __init__(
        self, 
        in_channels: int, out_channels: int, n_nodes: int, shrare_msg: bool=True,
        model_self: bool=True, share_model_self: bool=True,
        bias: bool=True, share_bias: bool=True
    ) -> None:
        """Intialize."""
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.n_nodes = n_nodes
        self.model_self = model_self
        super(GraphConv, self).__init__()
        
        # # message
        self.msg = VectorWiseLinear(
            in_channels, out_channels, n_nodes, False, shrare_msg)

        # # self-modeling
        if model_self:
            self.model_self = VectorWiseLinear(
                in_channels, out_channels, n_nodes, False, share_model_self)
        
        # # bias
        if bias:
            if share_bias:
                self.bias = nn.Parameter(torch.Tensor(out_channels))
            else:
                self.bias = nn.Parameter(torch.Tensor(n_nodes, out_channels))
            bound = 1 / np.sqrt(out_channels)
            nn.init.uniform_(self.bias, -bound, bound)
        else:
            self.register_parameter('bias', None)
            
    
    def forward(self, X: torch.Tensor, A: torch.Tensor, W: torch.Tensor=None) -> torch.Tensor:
        """Forward.
        
        Args:
            X: (batch_size, n_nodes, n_channels)
                Array which represents bags of vectors.
                X[:, i, :] are corresponded to feature vectors of node i.
            A: (batch_size, n_nodes, n_nodes)
                Array which represents adjacency matrices.
                A[:, i, j] are corresponded to weights (scalar) of edges from node j to node i.
            W: (batch_size, n_nodes, n_nodes)
                Array which represents weight matrices between nodes.
        """
        if W is not None:
            A = A * W  # shape: (batch_size, n_nodes, n_nodes)
        
        # # update message
        M = X  #  shape: (batch_size, n_nodes, in_channels)
        # # # send message
        M = self.msg(M)  # shape: (batch_size, n_nodes, out_channels)
        # # # aggregate
        M = torch.matmul(A, M)  # shape: (batch_size, n_nodes, out_channels)
            
        # # update node
        # # # self-modeling
        H = M
        if self.model_self:
            H = H + self.model_self(X)
        if self.bias is not None:
            H = H + self.bias
        
        return H

In [ ]:
class GCN(nn.Module):
    """GCN for Stacking."""
    
    # def __init__(
    #     self, n_classes: int,
    #     n_channels_list: tp.List[int],
    #     add_self_loop: bool=False,
    #     kwargs_head: tp.Dict={},
    # ):
    def __init__(self, num_features, num_targets, hidden_size, cfg):
        self.model_name = "GCN"
        n_classes = 1
        n_channels_list =  [5, 16, 16, 16, 16, 16,16]#[8, 16, 16, 16, 16, 16, 16,]
        add_self_loop = True
        kwargs_head = {
            "n_features_list": [-1, 768, 1],# [-1, 2048, 1]
            "use_tail_as_out": True,
            "drop_rate": 0.8,
            "use_bn": False,
            "use_wn": True,
            "block_name": "LABD",
        }
        
        
        super().__init__()
        self.n_conv_layers = len(n_channels_list) - 1
        for i in range(self.n_conv_layers):
            in_ch = n_channels_list[i]
            out_ch = n_channels_list[i + 1]
            # layer = CustomGraphConv(in_ch, out_ch, n_classes)
            layer = GraphConv(
                in_ch, out_ch, n_classes,
                shrare_msg=False, share_model_self=False, share_bias=False)
            setattr(self, "conv{}".format(i + 1), layer)
        
        self.relu = nn.ReLU(inplace=True)
        if add_self_loop:
            adj_mat = torch.ones(n_classes, n_classes) / n_classes
        else:
            adj_mat = (1 - torch.eye(n_classes, n_classes)) / (n_classes - 1) 
        self.register_buffer("A", adj_mat.float())
               
        kwargs_head["n_features_list"][0] = n_classes * n_channels_list[-1]
        self.head = MLP(**kwargs_head)
    
    def forward(self, X: torch.FloatTensor) -> torch.Tensor:
        """"""
        X = X.unsqueeze(1)
        bs, n_classes = X.shape[:2]
        H = X  # shape: (bs, n_classes, n_models)
        for i in range(self.n_conv_layers):
            H = getattr(self, "conv{}".format(i + 1))(H, self.A[None, ...])
            H = self.relu(H)
        
        h = torch.reshape(H, (bs, -1))
        h = self.head(h)
        return h

In [ ]:
stacking_CFG.model_path = stacking_CFG.model_dir+"gcn/"
predictions = np.zeros((len(test), 1))

SEED = [42, 1999, 2022]
device = torch.device("cpu")
for i, seed in enumerate(SEED):
    print(seed)
    _pred = run_kfold_nn(stacking_CFG, test_data, features, device,
                        n_fold=5, seed=seed, MODEL=GCN, save_dir=stacking_CFG.model_path,
                         )
    predictions += _pred/len(SEED)

predictions_gcn = predictions

# tabnet

In [ ]:
# os.system('python -m pip install --no-index --find-links=../input/pytorchtabnet/pytorch-tabnet  pytorch-tabnet')

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
stacking_CFG.model_path = stacking_CFG.model_dir+"tabnet/"
predictions = np.zeros((len(test), 1))

SEED = [42, 1999, 2022]
device = torch.device("cpu")
for i, seed in enumerate(SEED):
    print(seed)
    _pred = run_kfold_nn(stacking_CFG, test_data, features, device,
                        n_fold=5, seed=seed, MODEL=TabNetRegressor, save_dir=stacking_CFG.model_path, like_sklearn=True
                         )
    predictions += _pred/len(SEED)

In [ ]:
predictions_tabnet = predictions


In [ ]:
prediction_df = pd.DataFrame(np.concatenate([
    predictions_gcn,predictions_1dcnn, predictions_2dcnn, predictions_mlp, predictions_tabnet
], axis=1))

In [ ]:
final_predictions = []
for model in models:
    prediction = model.predict(prediction_df)
    final_predictions.append(prediction)
final_prediction = np.mean(final_predictions,axis=0)

# submission["score"] = final_prediction
# submission[["id", "score"]].to_csv(f"submission.csv",index=False)

final_prediction

# postprocess


In [ ]:
len(preds)

In [ ]:
# dataset
oof_df_exp020 = pd.read_pickle(os.path.join(CFG_exp020.path, "preds", "oof_df.pkl"))
postprocess_df = pd.merge(train_df[["id"]], oof_df_exp020, on="id", how="left")
postprocess_df = postprocess_df.drop(["pred", "fold", "score_map", "anchor_map"],axis=1)
postprocess_df["pred"] = preds


print(get_score(postprocess_df["score"], postprocess_df["pred"]))

## 残差を算出
postprocess_df["res"] = postprocess_df["score"] - postprocess_df["pred"]
postprocess_df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from contextlib import contextmanager
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
from sklearn.pipeline import Pipeline
import re
from bs4 import BeautifulSoup

class BaseBlock(object):
    def fit(self, input_df, y=None):
        return self.transform(input_df)
    
    def transform(self, input_df):
        raise NotImplementedError()

        
class ContinuousBlock(BaseBlock):
    def __init__(self, column):
        self.column = column
        
    def transform(self, input_df):
        return input_df[self.column].copy()


# 大元の実装では、whole_df(train_df+test_df)であるが、今回は裏側にいる
# これはいいのか？
class CountEncodingBlock(BaseBlock):
    def __init__(self, column, whole_df):
        self.column = column
        self.whole_df = whole_df
    
    def transform(self, input_df):
        output_df = pd.DataFrame()
        vc = self.whole_df[self.column].value_counts()
        output_df[self.column] = input_df[self.column].map(vc)
        
        return output_df.add_prefix("CE_")


class OneHotEncodingBlock(BaseBlock):
    def __init__(self, column, count_limit: int):
        self.column = column
        self.count_limit = count_limit

    def fit(self, input_df, y=None):
        vc = input_df[self.column].dropna().value_counts()
        cats_ = vc[vc > self.count_limit].index
        self.cats_ = cats_
        return self.transform(input_df)

    def transform(self, input_df):
        x = pd.Categorical(input_df[self.column], categories=self.cats_)
        output_df = pd.get_dummies(x, dummy_na=False)
        output_df.columns = output_df.columns.to_list()
        return output_df.add_prefix(f'OHE_{self.column}=')
    

class LabelEncodingBlock(BaseBlock):
    def __init__(self, column, whole_df):
        self.column = column
        self.whole_df = whole_df
        self.le = LabelEncoder()
        
    def fit(self, input_df, y=None):
        self.le.fit(self.whole_df[self.column].fillna("nan"))
        return self.transform(input_df)
    
    def transform(self, input_df):
        c = self.column
        output_df = input_df.copy()
        output_df[c] = self.le.transform(input_df[c].fillna("nan")).astype(int)
        output_df = output_df[[c]]
        return output_df.add_prefix("LE_")
    
    
    class WrapperBlock(BaseBlock):
        def __init__(self,function):
            self.function = function
            
        def transform(self, input_df):
            return self.function(input_df)


class AggregationBlock(BaseBlock):
    def __init__(self,
                 whole_df: pd.DataFrame,
                 key: str,
                 agg_column: str,
                 agg_funcs: ["mean"],
                 fillna=None):
        self.whole_df = whole_df
        self.key = key
        self.agg_column = agg_column
        self.agg_funcs = agg_funcs
        self.fillna = fillna

    def fit(self, input_df):
        return self.transform(input_df)

    def transform(self, input_df):
        if self.fillna:
            self.whole_df[self.agg_column] = self.whole_df[self.agg_column].fillna(self.fillna)

        self.group_df = self.whole_df.groupby(self.key).agg({self.agg_column: self.agg_funcs}).reset_index()
        column_names = [f'GP_{self.agg_column}@{self.key}_{agg_func}' for agg_func in self.agg_funcs]

        self.group_df.columns = [self.key] + column_names
        output_df = pd.merge(input_df[self.key], self.group_df, on=self.key, how="left").drop(columns=[self.key])
        return output_df


class StringLengthBlock(BaseBlock):
    def __init__(self, column):
        self.column = column

    def transform(self, input_df):
        output_df = pd.DataFrame()
        output_df[self.column] = input_df[self.column].str.len()
        return output_df.add_prefix("StringLength_")


class WordCountBlock(BaseBlock):
    def __init__(self, column: str):
        self.column = column

    def transform(self, input_df):
        output_df = pd.DataFrame()
        output_df[f"WORDCOUNT_{self.column}"] = input_df[self.column].astype(str).map(lambda x: len(x.split()))

        return output_df
    

def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text

    
class TfidfBlock(BaseBlock):
    def __init__(self, column: str, whole_df: pd.DataFrame, decomposition: str, n_compose: int):
        self.column = column
        self.whole_df = whole_df
        self.decomposition = decomposition
        self.n_compose = n_compose

    def fit(self, input_df, y=None):
        master_df = self.whole_df
        text = self.whole_df[self.column].fillna("")

        if self.decomposition == "svd":
            self.pipeline_ = Pipeline([
                ("tfidf", TfidfVectorizer(max_features=50000)),
                ("svd", TruncatedSVD(n_components=self.n_compose, random_state=71))
            ])

        elif self.decomposition == "NMF":
            self.pipeline_ = Pipeline([
                ("tfidf", TfidfVectorizer(max_features=50000)),
                ("NMF", NMF(n_components=self.n_compose, random_state=71))
            ])
        elif self.decomposition == "LDA":
            self.pipeline_ = Pipeline([
                ("tfidf", TfidfVectorizer(max_features=50000)),
                ("NMF", LatentDirichletAllocation(n_components=self.n_compose, random_state=71))
            ])
        else:
            self.pipeline_ = Pipeline([
                ("tfidf", TfidfVectorizer(max_features=50000)),
                ("svd", TruncatedSVD(n_components=self.n_compose, random_state=71))
            ])

        self.pipeline_.fit(text)

        return self.transform(input_df)

    def transform(self, input_df):
        text = input_df[self.column].fillna("")
        z = self.pipeline_.transform(text)

        out_df = pd.DataFrame(z)
        return out_df.add_prefix(f'{self.column}_tfidf_{self.decomposition}_')
    

In [ ]:
@contextmanager
def timer(logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None):
    if prefix: format_str = str(prefix) + format_str
    if suffix: format_str = format_str + str(suffix)
    start = time()
    yield
    d = time() - start
    out_str = format_str.format(d)
    if logger:
        logger.info(out_str)
    else:
        print(out_str)


def get_function(block, is_train):
    s = mapping = {
        True: 'fit',
        False: 'transform'
    }.get(is_train)
    return getattr(block, s)


def to_feature(input_df,
               blocks,
               is_train=False):
    out_df = pd.DataFrame()

    for block in tqdm(blocks, total=len(blocks)):
        func = get_function(block, is_train)

        with timer(prefix='create ' + str(block) + ' '):
            _df = func(input_df)
        assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)
    return reduce_mem_usage(out_df)


def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        if verbose:
            print(col)
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'
              .format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
MODEL_NAME = '../input/uspppm-debertv3large-5folds/uspppm_0'

# トークナイザとモデルのロード
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
#tokenizer = DebertaV2TokenizerFast.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
model = model.cuda()

In [ ]:
## testデータの読み込み
_test_df = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/test.csv")
titles = pd.read_csv('../input/cpc-codes/titles.csv')
_test_df = _test_df.merge(titles, left_on='context', right_on='code')
_test_df.head()

In [ ]:
_test_df['input'] = _test_df['title']+' '+_test_df['anchor']
_test_df['input2'] = _test_df["input"] + " " + _test_df["target"]

max_length = 128
sentence_vectors = []

for n in range(len(_test_df)):
    # 記事から文章を抜き出し、符号化を行う。
    text = _test_df["input2"][n]
    encoding = tokenizer(
        text, 
        max_length=max_length, 
        padding='max_length', 
        truncation=True, 
        return_tensors='pt'
    )
    encoding = { k: v.cuda() for k, v in encoding.items() } 
    attention_mask = encoding['attention_mask']

    # 文章ベクトルを計算
    # BERTの最終層の出力を平均を計算する。（ただし、[PAD]は除く。）
    with torch.no_grad():
        output = model(**encoding)
        last_hidden_state = output.last_hidden_state 
        averaged_hidden_state = \
            (last_hidden_state*attention_mask.unsqueeze(-1)).sum(1) \
            / attention_mask.sum(1, keepdim=True) 

    # 文章ベクトルとラベルを追加
    sentence_vectors.append(averaged_hidden_state[0].cpu().numpy())

sentence_vectors = np.vstack(sentence_vectors)
sentence_vectors

In [ ]:
deberta_emb_ts = pd.DataFrame(sentence_vectors)
deberta_emb_ts = deberta_emb_ts.add_prefix("deberta_v3_large_")

In [ ]:
deberta_emb_ts = pd.concat([_test_df[["id"]], deberta_emb_ts],axis=1)
test = pd.merge(test,deberta_emb_ts,on="id", how="left")
test.head()

In [ ]:
postprocess_df.head(1)

In [ ]:
# 後処理用に新規でdfを起こしておく
new_train = postprocess_df.drop("text", axis=1)
new_test = test.drop("text", axis=1)

new_test["pred"] = final_prediction

# 前処理（context）
new_train["context2"] = new_train["context"].str[0]
new_train["context3"] = new_train["context"].str[1:].astype(str)

new_test["context2"] = new_test["context"].str[0]
new_test["context3"] = new_test["context"].str[1:].astype(str)

whole_df = pd.concat([new_train, new_test])

In [ ]:
new_train.info()

In [ ]:
process_blocks = [
    *[ContinuousBlock(c) for c in [
        "pred",
    ]],
    *[CountEncodingBlock(c, whole_df=whole_df) for c in [
        "context",
        "context2",
        "context3"
    ]],
    *[LabelEncodingBlock(c, whole_df=whole_df) for c in [
        "context"
    ]],
    *[StringLengthBlock(c) for c in [
        "anchor",
        "target",
        "context_text"
    ]],
    *[WordCountBlock(c) for c in [
        "anchor",
        "target",
        "context_text"
    ]],
    TfidfBlock("context_text" ,whole_df=whole_df, decomposition="svd", n_compose=150)
    ]

# 今回は残差が目的変数
train_y = postprocess_df["res"]
train_x = to_feature(new_train, process_blocks, is_train=True)
test_x = to_feature(new_test, process_blocks)

In [ ]:
train_x.head(1)

In [ ]:
test_x.head(1)

In [ ]:
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

def fit_lgbm(X, y, cv, params=None, verbose: int=50):
    metric_func = mean_squared_error
    
    if params is None:
        params = dict()
        
    models = []
    
    oof_pred = np.zeros_like(y, dtype=np.float)
    
    for i, (tr_idx, val_idx) in enumerate(cv):
        x_train, y_train = X[tr_idx], y[tr_idx]
        x_valid, y_valid = X[val_idx], y[val_idx]
        
        clf = lgb.LGBMRegressor(**params)
        
        with timer(prefix="fit_fold{}".format(i+1)):
            clf.fit(x_train, y_train,
                   eval_set=[(x_valid,y_valid)],
                   early_stopping_rounds=verbose,
                   verbose=-1)
        
        pred_i = clf.predict(x_valid)
        oof_pred[val_idx] = pred_i
        models.append(clf)
        
        print(f'Fold {i} RMSE: {metric_func(y_valid, pred_i)**.5 :.4f}')
    score = metric_func(y, oof_pred)**.5 
    print('FINISHED | Whole RMSE: {:.4f}'.format(score))
    return oof_pred, models

In [ ]:
from sklearn.model_selection import StratifiedGroupKFold
lgm_params = {  
    "n_estimators": 20000,
    "objective": 'rmse',
    "learning_rate": 0.01,
    "num_leaves": 32,
    "random_state": 71,
    "n_jobs": -1,
    "importance_type": "gain",
    'colsample_bytree': 0.7,
    "max_depth":5,
    }

postprocess_df['score_map'] = postprocess_df['score'].map({0.00: 0, 0.25: 1, 0.50: 2, 0.75: 3, 1.00: 4})
encoder = LabelEncoder()
postprocess_df['anchor_map'] = encoder.fit_transform(postprocess_df['anchor'])

fold_lgbm = Fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=71)
cv_lgbm = list(fold_lgbm.split(postprocess_df, postprocess_df['score_map']))

oof_lgbm, models_lgbm = fit_lgbm(train_x.values, train_y, cv_lgbm , params=lgm_params)

In [ ]:
# 追加後処理
# https://www.kaggle.com/competitions/us-patent-phrase-to-phrase-matching/discussion/327288
# 意味なかったので使用せず
def update_label(pred):
    if -0.01 < pred <= .01:
        return .0
    elif .24 <= pred <= .26:
        return .25
    elif .49 <= pred <= .51:
        return .5
    elif .74 <= pred <= .76:
        return .75
    elif .99 <= pred <= 1.01:
        return 1.0

    return pred

In [ ]:
# 出力結果が補正値になるので、これを結果に当てはめる
postprocess_df["res_oof"] = oof_lgbm
postprocess_df["pp_pred"] = postprocess_df["res_oof"] + postprocess_df["pred"]


before_pp = get_score(postprocess_df["score"], postprocess_df["pred"])
after_pp = get_score(postprocess_df["score"], postprocess_df["pp_pred"])
print(f"before_pp：{round(before_pp,4)}")
print(f"after_pp：{round(after_pp,4)}")

In [ ]:
fig = plt.figure(figsize=(12,4))

ax1 = fig.add_subplot(1, 2, 1)
sns.scatterplot(postprocess_df["score"], postprocess_df["pred"],ax=ax1)
ax1.set_ylim(-0.1, 1.1)

ax2 = fig.add_subplot(1, 2, 2)
sns.scatterplot(postprocess_df["score"], postprocess_df["pp_pred"],ax=ax2)
ax2.set_ylim(-0.1, 1.1)

In [ ]:
def visualize_importance(models, feat_train_df):
    """
    args:
        models:
            List of lightGBM models
        feat_train_df:
            学習時に使った DataFrame
    """
    feature_importance_df = pd.DataFrame()
    for i, model in enumerate(models):
        _df = pd.DataFrame()
        _df['feature_importance'] = model.feature_importances_
        _df['column'] = feat_train_df.columns
        _df['fold'] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, _df], 
                                          axis=0, ignore_index=True)

    order = feature_importance_df.groupby('column')\
        .sum()[['feature_importance']]\
        .sort_values('feature_importance', ascending=False).index[:50]

    fig, ax = plt.subplots(figsize=(8, max(6, len(order) * .25)))
    sns.boxenplot(data=feature_importance_df, 
                  x='feature_importance', 
                  y='column', 
                  order=order, 
                  ax=ax, 
                  palette='viridis', 
                  orient='h')
    ax.tick_params(axis='x', rotation=90)
    ax.set_title('Importance')
    ax.grid()
    fig.tight_layout()
    return fig, ax

In [ ]:
fig, ax = visualize_importance(models_lgbm, train_x)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
sns.histplot(train_y, label='Train_y', ax=ax, color='black')
sns.histplot(oof_lgbm, label='Out Of Fold', ax=ax, color='C1')
ax.legend()
ax.grid()

In [ ]:
final_prediction_pp = np.array([model.predict(test_x.values) for model in models_lgbm])
final_prediction_pp = np.mean(final_prediction_pp, axis=0)

In [ ]:
#final_prediction = np.where(final_prediction < 0, 0, final_prediction)
#final_prediction = np.where(final_prediction > 1, 1, final_prediction)
submission['score'] = final_prediction + final_prediction_pp

# post process2
same_ids = test[test["anchor"]==test["target"]]["id"].tolist()
if len(same_ids) > 0:
    submission.loc[submission["id"].isin(same_ids), "score"] = 1.0 # 同じphaseが入っているものは1に変換する
    
display(submission.head())
submission[['id', 'score']].to_csv('submission.csv', index=False)